# Libraries

In [19]:
import pandas as pd
import numpy as np

from ethanol_brazil_plants.utils import utils

from datetime import date
from xlrd.xldate import xldate_as_datetime

# Functions

In [20]:
def normalize_renovabio_database(
    raw_df: pd.DataFrame,
    object_cols: list[str],
    int_cols: list[str],
    float_cols: list[str],
    date_cols: list[str],
) -> pd.DataFrame:
    """
    Normalize renovabio database columns
    """

    normalized_df = utils.normalize_header(raw_df)
    normalized_df = normalized_df.fillna(method='ffill')

    normalized_df = utils.normalize_object_columns(normalized_df, object_cols)
    normalized_df = normalized_df[normalized_df['CNPJ'] != '']

    normalized_df = utils.normalize_int_columns(normalized_df, int_cols)
    normalized_df = utils.normalize_float_columns(normalized_df, float_cols)

    normalized_df = utils.normalize_date_columns(normalized_df, date_cols)

    return normalized_df

In [21]:
def generate_razao_cidade_uf(normalized_df: pd.DataFrame) -> pd.DataFrame:
    """
    Generate RAZAO_SOCIAL, CIDADE and UF columns.
    """

    normalized_df['RAZAO_SOCIAL'] = normalized_df['RAZAO_SOCIAL-CIDADE-UF'].str.split('-').str[0]

    normalized_df[['CIDADE', 'UF', 'UF_2']] = (
        normalized_df['RAZAO_SOCIAL-CIDADE-UF']
        .str.split('-')
        .str[-2:]
        .str.join(',')
        .str.split(',', expand=True)
    )

    normalized_df['UF'] = np.where(
        normalized_df['UF'].isnull(),
        normalized_df['UF_2'],
        normalized_df['UF']
    )

    normalized_df['CIDADE'] = np.where(
        normalized_df['RAZAO_SOCIAL'] !=
        normalized_df['CIDADE'],
        normalized_df['CIDADE'],
        None
    )

    normalized_df = normalized_df.drop(['RAZAO_SOCIAL-CIDADE-UF', 'UF_2'], axis=1)
    return normalized_df

# Data

In [22]:
raw_rbp_validos = catalog.load("raw_renovabio_plants_validos")

[08/05/23 21:12:02] INFO     Loading data from 'raw_renovabio_plants_validos' (ExcelDataSet)... ]8;id=872508;file://C:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=113003;file://C:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

# Normalize

In [23]:
# Normalize
object_cols = [
    'RAZAO_SOCIAL-CIDADE-UF',
    'BIOCOMBUSTIVEL',
    'ROTA',
    'FIRMA_INSPETORA',
    'ENDERECO_EMISSOR_PRIMARIO',
    'CNPJ',
    'PROCESSO_DE_CERTIFICACAO',
]

int_cols = [
    'CNPJ',
    'PROCESSO_DE_CERTIFICACAO']
    
float_cols = [
    'NOTA_DE_EFICIENCIA_ENERGETICO-AMBIENTAL_GCO2EQ/MJ',
    'VOLUME_ELEGIVEL_%',
    'FATOR_PARA_EMISSAO_DE_CBIO_TCO2EQ/L',
]

date_cols = ['DATA_DE_APROVACAO_PELA_ANP', 'VALIDADE']

normalized_rbp_validos = normalize_renovabio_database(
    raw_rbp_validos, object_cols,
    int_cols, float_cols, date_cols,
)

# RAZAO_SOCIAL, CIDADE and UF

In [24]:
normalized_rbp_validos = generate_razao_cidade_uf(normalized_rbp_validos)

# Rota

In [25]:
ds_rota = {
    'ETANOL COMBUSTIVEL DE PRIMEIRA GERACAO CANA-DE-ACUCAR':
    'ETANOL 1GEN CANA',
    'ETANOL COMBUSTIVEL DE PRIMEIRA GERACAO CANA-DE-ACUCAR E MILHO EM USINA INTEGRADA':
    'ETANOL 1GEN CANA E MILHO',
    'ETANOL COMBUSTIVEL DE PRIMEIRA GERACAO MILHO':
    'ETANOL 1GEN MILHO',
    'ETANOL COMBUSTIVEL DE PRIMEIRA E SEGUNDA GERACAO PRODUZIDO EM USINA INTEGRADA':
    'ETANOL 1GEN E 2GEN',
}

cd_rota = {
    'BIODIESEL': 1,
    'BIOMETANO': 2,
    'ETANOL 1GEN E 2GEN': 3,
    'ETANOL 1GEN CANA': 4,
    'ETANOL 1GEN CANA E MILHO': 5,
    'ETANOL 1GEN MILHO': 6,
}

normalized_rbp_validos['DS_ROTA'] = normalized_rbp_validos['ROTA'].replace(ds_rota)
normalized_rbp_validos['CD_ROTA'] = normalized_rbp_validos['DS_ROTA'].replace(cd_rota)

# Location

## CEP

In [26]:
normalized_rbp_validos['CEP'] = (
    normalized_rbp_validos['ENDERECO_EMISSOR_PRIMARIO']
    .str.replace('.','')
    .str.replace(r'-(\s+)', '-', regex=True)
    .str.findall(r'\d{5}-\d{3}|\d{5}-\d{2}')
    .str[0]
    .str.replace('-','')
    .str.pad(8, side='right', fillchar='0')
)